In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import torch 
from PIL import Image 
import cv2
import albumentations as A
from matplotlib import patches, pyplot as plt 
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Let's Look at our Data**

In [ ]:
DATA_DIR = "../input/nfl-impact-detection/"
im_path = "../input/nfl-impact-detection/images/"

In [ ]:
train_label = pd.read_csv("../input/nfl-impact-detection/train_labels.csv")
image_label = pd.read_csv(DATA_DIR+"image_labels.csv")
train_player_tracking = pd.read_csv(DATA_DIR+"train_player_tracking.csv")

In [ ]:
train_label.head(2)

In [ ]:
image_label.head(2)
len(image_label["image"].unique())

In [ ]:
train_player_tracking.head(2)

In [ ]:
len(image_label["label"].unique())

In [ ]:
len(train_label["label"].unique())

In [ ]:

def draw_bb(ax, img, im_df):
    
    img_data= im_df[im_df["image"]==img]
    #print(img_data.shape)
    for i in range(img_data.shape[0]):
        data = img_data.iloc[i]
        bb = patches.Rectangle(
            (data["left"], 
            data["top"]),
            data["width"], 
            data["height"],
            linewidth =2, 
            edgecolor = "red"
        )
        ax.add_patch(bb)
            
    return 
    
def plot_img_with_bb():  
    
   
    #image_label = pd.read_csv(DATA_DIR+"image_labels.csv")
    img_list = os.listdir(im_path)
    
    fig, ax = plt.subplots(3,3, figsize=(16,14))
    for i in range(3):
        for j in range (3):
            
            #plt.subplot(3,3,i+1)
            rand_idx = np.random.randint(len(img_list))
            img =Image.open(im_path+img_list[rand_idx])
            #print(img)
            ax[i][j].imshow(img)
            
            draw_bb(ax[i][j], img_list[rand_idx], image_label)

            ax[i][j].set_xticklabels([])
            ax[i][j].set_yticklabels([])
        fig.show()
    fig.subplots_adjust(wspace=0, hspace=0)


In [ ]:
#img_list = image_label["image"].unique()
plot_img_with_bb()

# Model Faster RCNN 

In [ ]:

class NFLDataset(object):
    def __init__(self, df, image_dir):
    
        self.image_ids = df['image'].unique()
        self.df = df
        self.image_dir = image_dir
        #self.transforms = transforms
        self.labels_dict = {
            'Helmet':1, 
            'Helmet-Blurred':2, 
            'Helmet-Difficult':3, 
            'Helmet-Sideline':4,
            'Helmet-Partial':5
        }

    def __getitem__(self, idx:int):
        # load images ad masks
        image_id = self.image_ids[idx]
        image = np.array(Image.open(f'{self.image_dir}/{image_id}'))/225.0
        image = np.moveaxis(image, 2, 0)
        records = self.df[self.df["image"]==self.image_ids[idx]]
        boxes = []
        labels = []
        for i in range(records.shape[0]):
            img_data = records.iloc[i]
            x_min = img_data.left
            x_max = img_data.left + img_data.width
            y_min = img_data.top
            y_max = img_data.top + img_data.height
            boxes.append([x_min, y_min, x_max, y_max])
            label = self.labels_dict[img_data.label]
            labels.append(label)

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there are 5 classes 
        labels = torch.as_tensor(labels, dtype=torch.int64)
        image  = torch.as_tensor(image , dtype = torch.float32)
        image_id = torch.tensor([idx])
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        
        '''if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
'''
        return image, target, image_id

    def __len__(self):
        return self.image_ids.shape[0]

In [ ]:
#albumentation 
def get_train_transform():
    return A.Compose([
        A.Flip(0.5),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 6  
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


In [ ]:
#plt.imshow(Image.open(os.path.join(im_path, image_label["image"][0])).convert("RGB"))


In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))


train_dataset = NFLDataset(image_label, im_path)
valid_dataset = NFLDataset(image_label, im_path)
# split the dataset in train and test set
#indices = torch.randperm(len(dataset)).tolist()
indices = torch.randperm(len(train_dataset)).tolist()
train_cnt = int(0.9*len(indices))

train_dataset = torch.utils.data.Subset(train_dataset,indices[:train_cnt])
valid_dataset = torch.utils.data.Subset(valid_dataset,indices[train_cnt:])



train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_fn
)

valid_data_loader =  torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_fn
)


In [ ]:
#print(len(indices), len(train_dataset), len(valid_dataset))

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
images, targets, image_ids = next(iter(train_data_loader))
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

In [ ]:
boxes = targets[0]['boxes'].cpu().numpy().astype(np.int32)
sample = images[0].permute(1,2,0).cpu().numpy()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)

# Define and Train Model

In [ ]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
lr_scheduler = None

num_epochs = 2


In [ ]:

#torch.cuda.empty_cache()
loss = []
iterations = []
for epoch in range(num_epochs):
    

    for i, batch in enumerate(train_data_loader):
        if torch.cuda.is_available():
            images, targets, image_ids = batch
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            model.train()
            loss_dict = model(images, targets)

            losses = sum(loss for loss in loss_dict.values())
            loss_value = losses.item()

            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
            
            if i % 1000 == 0:
                print(f"Iteration #{i} loss: {loss_value}")
            
            loss.append(loss_value)
            iterations.append(i)
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()
        
    plt.plot(iterations, loss)
    plt.show()


In [ ]:
torch.save(model.state_dict(), "frcnnresnet50.pth")

**Test**

In [ ]:
images, targets, image_ids = next(iter(valid_data_loader))
images = list(img.to(device) for img in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
boxes = targets[0]['boxes'].cpu().numpy().astype(np.int32)
sample = images[0].permute(1,2,0).cpu().numpy()
model.eval()
cpu_device = torch.device("cpu")

outputs = model(images)
outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)